In [1]:
# ssh -L 9000:localhost:9000 thanrada@10.121.101.110
# http://127.0.0.1:9000/?token=3fa074da2984241394d534e672b246eff7b82102c9b6f317
# https://iservice.dopa.go.th/web/map/license/map/

In [2]:
import pandas as pd
import numpy as np

In [3]:
import subprocess
import time
import os
from IPython.display import display, Markdown

# --- 1. ตรวจสอบและแสดงสถานะ GPU ---
# **คำแนะนำ:** กรุณาตรวจสอบให้แน่ใจว่าคุณได้เลือก "Runtime type" เป็น GPU (เช่น T4, A100 หรือ V100) แล้ว
display(Markdown("## 1. ตรวจสอบสถานะ GPU"))
try:
    print("Running nvidia-smi to check GPU status...")
    subprocess.run("nvidia-smi", shell=True, check=True)
except subprocess.CalledProcessError:
    print("\n[🚨 WARNING] Could not run nvidia-smi. Please ensure your Colab runtime is set to GPU.")

# --- 2. ติดตั้งเครื่องมือและ Ollama ---
display(Markdown("\n## 2. ติดตั้ง Ollama และเครื่องมือที่จำเป็น"))

# ติดตั้ง lshw ตามที่ร้องขอ
print("Installing lshw...")
subprocess.run("apt-get install -y lshw > /dev/null 2>&1", shell=True, check=True)
subprocess.run("lshw -C display", shell=True)

# ติดตั้ง Ollama
print("\nInstalling Ollama...")
# สคริปต์ติดตั้ง Ollama
ollama_install_script = """
curl -fsSL https://ollama.com/install.sh | sh
"""
subprocess.run(ollama_install_script, shell=True, check=True)
print("Ollama installed successfully.")

# --- 3. เริ่มต้น Ollama Server และรอให้พร้อมใช้งาน ---
display(Markdown("\n## 3. เริ่มต้น Ollama Server"))

# รัน Ollama server ใน background
# ใช้ nohup และ & เพื่อรันเป็น background process ใน Colab
print("Starting Ollama server in background...")
os.system("nohup ollama serve > ollama.log 2>&1 &")

# ให้เวลากับ server ในการเริ่มต้น
print("Waiting 15 seconds for Ollama server to initialize...")
time.sleep(15)

# ตรวจสอบว่า server รันอยู่หรือไม่
try:
    print("\nChecking server status...")
    subprocess.run("ollama list", shell=True, check=True)
    print("Ollama server is running and accessible.")
except subprocess.CalledProcessError:
    print("\n[🚨 ERROR] Ollama server failed to start or respond. Check 'ollama.log'.")
    exit()

# --- 4. ดึง (Pull) โมเดลที่ต้องการ ---
display(Markdown("\n## 4. ดึง (Pull) โมเดล Llama3 และ Typhoon"))

# ดึง Llama3
print("\nPulling llama3 (ประมาณ 4.7 GB)...")
subprocess.run("ollama pull llama3", shell=True, check=True)

# ดึง scb10x/typhoon2.1-gemma3-4b:latest (โมเดลภาษาไทย)
print("\nPulling scb10x/typhoon2.1-gemma3-4b:latest...")
subprocess.run("ollama pull scb10x/typhoon2.1-gemma3-4b:latest", shell=True, check=True)

display(Markdown("\n## 5. ทดสอบการรันโมเดล"))
print("--- Running test prompt with llama3 ---")
# ทดสอบรันโมเดลเพื่อให้แน่ใจว่าใช้งานได้
test_prompt = "Explain why the sky is blue in one sentence."
try:
    # ใช้ -q เพื่อไม่แสดงข้อมูลระหว่างการดาวน์โหลด/รัน
    subprocess.run(f'ollama run llama3 "{test_prompt}"', shell=True, check=True)
    print("\n✅ Setup Complete! Both models are pulled and ready for use.")
except subprocess.CalledProcessError as e:
    print(f"\n[🚨 FATAL ERROR] Test run failed. Error: {e}")

display(Markdown("\n---"))
display(Markdown("### วิธีการใช้งานต่อ:"))
display(Markdown("คุณสามารถเรียกใช้โมเดลในเซลล์ใหม่ได้ทันที เช่น:"))
display(Markdown("`!ollama run llama3 \"What is the capital of France?\"`"))
display(Markdown("`!ollama run scb10x/typhoon2.1-gemma3-4b:latest \"เขียนบทกลอนสั้น ๆ เกี่ยวกับประเทศไทย\"`"))


## 1. ตรวจสอบสถานะ GPU

Running nvidia-smi to check GPU status...

[🚨 WARNING] Could not run nvidia-smi. Please ensure your Colab runtime is set to GPU.



## 2. ติดตั้ง Ollama และเครื่องมือที่จำเป็น

Installing lshw...

Installing Ollama...
Ollama installed successfully.



## 3. เริ่มต้น Ollama Server

...


---

### วิธีการใช้งานต่อ:

คุณสามารถเรียกใช้โมเดลในเซลล์ใหม่ได้ทันที เช่น:

`!ollama run llama3 "What is the capital of France?"`

`!ollama run scb10x/typhoon2.1-gemma3-4b:latest "เขียนบทกลอนสั้น ๆ เกี่ยวกับประเทศไทย"`

In [4]:
hotel = pd.read_csv('https://raw.githubusercontent.com/Thanrada2024/illegal_accommodation/refs/heads/main/bora/hotel_bangkok.csv').iloc[:,[0,1,3,4]]
hotel.columns = ['Hotel_Ref','Hotel_Name','Hotel_Address','N']
hotel["Hotel_Address"]= hotel["Hotel_Address"].replace("บ้านเลขที่ ","", regex=True)
hotel.head()

,Hotel_Ref,Hotel_Name,Hotel_Address,N
0,101/2563,โรงแรม สุทธิสารพ้อยส์,45 ซ.ยิ้มอุปถัมภ์ แขวงรัชดาภิเษก เขตดินแดง จ.ก...,42
1,136/2563,โรงแรม คณาบูทีค 2,"3,5,7,9,11 ซ.จรัญสนิทวงศ์ 57/2 แขวงบางบำหรุ เข...",24
2,152/2563,วัลย์ลดา เพลส,71/1 ซ.รามคำแหง 53 (จันทร์ศรีชวาลา) แขวงพลับพล...,50
3,214/2563,โรงแรม นิวสยาม พาเลซวิลล์ 2,56 ตรอกโรงไหม แขวงชนะสงคราม เขตพระนคร จ.กรุงเท...,48
4,215/2563,โรงแรม นิวสยาม พาเลซวิลล์ 3,58 ตรอกโรงไหม แขวงชนะสงคราม เขตพระนคร จ.กรุงเท...,48


In [30]:
trip_com = pd.read_csv('https://raw.githubusercontent.com/Thanrada2024/illegal_accommodation/refs/heads/main/OTA/trip_com_bkk.csv')[['name','name href','address']].dropna()
trip_com['OTA'] = 'trip_com'

booking_com = pd.read_csv('https://raw.githubusercontent.com/Thanrada2024/illegal_accommodation/main/OTA/booking_com_bkk_with_addr.csv', engine='python')[['name href','name','address']].dropna()
booking_com['OTA'] = 'booking_com'

# รวมสอง DataFrame เข้าด้วยกัน
# OTA_df = pd.concat([trip_com, booking_com], ignore_index=True)
OTA_df = pd.concat([trip_com], ignore_index=True).sample(500,random_state=42)
OTA_df.head()

,name,name href,address,OTA
31,โรงแรมคิวว์ สุขุมวิท,https://th.trip.com/hotels/detail/?cityId=359&...,"9 สุขุมวิท 79 พระโขนงเหนือ, วัฒนา, เขตวัฒนา, ก...",trip_com
413,โรงแรมสำราญเพลส,https://th.trip.com/hotels/detail/?cityId=359&...,"302 ถ., เขตราชเทวี, กรุงเทพฯ, 10400",trip_com
536,ไอบิส สไตล์ กรุงเทพ สุขุมวิท 4,https://th.trip.com/hotels/detail/?cityId=359&...,"27 ซ.สุขุมวิท 4 คลองเตย, เขตคลองเตย, กรุงเทพฯ,...",trip_com
745,ทู ทรี อะ โฮมลี่ โฮเทล,https://th.trip.com/hotels/detail/?cityId=359&...,"100 ซ. สุขุมวิท 23 แขวงคลองเตยเหนือ, เขตวัฒนา,...",trip_com
877,โรงแรมกลับ,https://th.trip.com/hotels/detail/?cityId=359&...,"22/1 ถ.เพชรบุรี ซ.13, เขตราชเทวี, กรุงเทพฯ, 10400",trip_com


## Method 1: Semantic search

### mrp/simcse-model-m-bert-thai-cased

In [31]:
# ---------- ตั้งค่า (from 614a1f16) ----------
import re, unicodedata
import numpy as np
import pandas as pd
from sentence_transformers import SentenceTransformer
from sklearn.neighbors import NearestNeighbors
from tqdm.auto import tqdm
from sklearn.metrics.pairwise import cosine_similarity

# =========================
# 1) Clean: ชื่อ/ที่อยู่ + สร้าง "คีย์" name+address
# =========================
def normalize_name(s: str) -> str:
    """ตัดคำ 'โรงแรม/Hotel' และ 'กรุงเทพ/กรุงเทพฯ/กรุงเทพมหานคร' + ลบเว้นวรรค/สัญลักษณ์"""
    if pd.isna(s): return ""
    s = unicodedata.normalize("NFKC", str(s)).lower()
    s = re.sub(r"(โรงแรม|hotel| โฮเทล|โฮเทล)", " ", s, flags=re.IGNORECASE)
    s = re.sub(r"กรุงเทพ(?:มหานคร|ฯ)?", " ", s)  # ตัดคำ 'กรุงเทพ' ออกจาก 'ชื่อ'
    s = re.sub(r"[,\|\(\)\[\]{}\"'’`•·/.:;#&\-–—_+*^@!$?~]", " ", s)
    s = re.sub(r"\s+", "", s)  # ลบช่องว่างทั้งหมด
    return s

def normalize_addr(s: str) -> str:
    """คงโครงสร้างที่อยู่ไว้ (ไม่ลบช่องว่างทั้งหมด) แต่ normalize ให้เท่า ๆ กัน"""
    if pd.isna(s): return ""
    s = unicodedata.normalize("NFKC", str(s)).lower().strip()

    # ทำให้คำหลักสอดคล้องกัน
    rep = {
        "กทม.": "กรุงเทพมหานคร",
        "กรุงเทพฯ": "กรุงเทพมหานคร",
        "จ.": "",              # จ.กรุงเทพมหานคร -> กรุงเทพมหานคร
        "เขต ": "",
        "แขวง ": "",
        "ถ.": "",
        "ถนน ": "",        # กันกรณีมีช่องว่างเพี้ยน
        "บ้านเลขที่": "",
    }
    for k,v in rep.items():
        s = s.replace(k, v)

    # เก็บตัวอักษร/ตัวเลข/เว้นวรรคและเครื่องหมาย / , - บางส่วน
    s = re.sub(r"[\"'’`•·\|\[\]{}#@!$^*?_+=:;,]", " ", s)
    s = re.sub(r"\s+", " ", s).strip()
    return "".join(s.split())

# apply clean
OTA_df["Trip_Name_n"]    = OTA_df["name"].map(normalize_name)
OTA_df["Trip_Address_n"] = OTA_df["address"].map(normalize_addr)
hotel["Hotel_Name_n"]    = hotel["Hotel_Name"].map(normalize_name)
hotel["Hotel_Address_n"] = hotel["Hotel_Address"].map(normalize_addr)

# คีย์ค้นหา = name + address (ตามที่ต้องการ)
OTA_df["Trip_Key"]   = (OTA_df["Trip_Name_n"]   + " " + OTA_df["Trip_Address_n"]).str.strip()
hotel["Hotel_Key"] = (hotel["Hotel_Name_n"] + " " + hotel["Hotel_Address_n"]).str.strip()

In [32]:
OTA_df[OTA_df['Trip_Name_n'].str.contains('กลับ', na=False)]

,name,name href,address,OTA,Trip_Name_n,Trip_Address_n,Trip_Key
877,โรงแรมกลับ,https://th.trip.com/hotels/detail/?cityId=359&...,"22/1 ถ.เพชรบุรี ซ.13, เขตราชเทวี, กรุงเทพฯ, 10400",trip_com,กลับ,22/1เพชรบุรีซ.13เขตราชเทวีกรุงเทพมหานคร10400,กลับ 22/1เพชรบุรีซ.13เขตราชเทวีกรุงเทพมหานคร10400


In [33]:
hotel[hotel['Hotel_Name'].str.contains('กลับ', na=False)]

,Hotel_Ref,Hotel_Name,Hotel_Address,N,Hotel_Name_n,Hotel_Address_n,Hotel_Key
450,141/2565,กลับโฮเทล,22/1 ซ.สมประสงค์ 2 แขวงถนนพญาไท เขตราชเทวี จ.ก...,79,กลับ,22/1ซ.สมประสงค์2แขวงถนนพญาไทเขตราชเทวีกรุงเทพม...,กลับ 22/1ซ.สมประสงค์2แขวงถนนพญาไทเขตราชเทวีกรุ...


In [43]:
# 0) ONE SOURCE OF TRUTH
OTA_df = OTA_df.reset_index(drop=True).copy()
hotel  = hotel.reset_index(drop=True).copy()

# 1) ensure columns exist & fillna
for col in ["Trip_Name_n","Trip_Address_n"]:
    if col not in OTA_df.columns:
        raise ValueError(f"ไม่พบคอลัมน์ {col} ใน OTA_df — ต้อง normalize ให้เรียบร้อยก่อน")
for col in ["Hotel_Name_n","Hotel_Address_n"]:
    if col not in hotel.columns:
        raise ValueError(f"ไม่พบคอลัมน์ {col} ใน hotel — ต้อง normalize ให้เรียบร้อยก่อน")

OTA_df["Trip_Name_n"]    = OTA_df["Trip_Name_n"].fillna("").astype(str)
OTA_df["Trip_Address_n"] = OTA_df["Trip_Address_n"].fillna("").astype(str)
hotel["Hotel_Name_n"]    = hotel["Hotel_Name_n"].fillna("").astype(str)
hotel["Hotel_Address_n"] = hotel["Hotel_Address_n"].fillna("").astype(str)

# 2) สร้างคีย์รวม (ถ้ายังไม่มี/อยากทับ)
OTA_df["Trip_Key"] = (OTA_df["Trip_Name_n"] + " " + OTA_df["Trip_Address_n"]).str.strip()
hotel["Hotel_Key"] = (hotel["Hotel_Name_n"] + " " + hotel["Hotel_Address_n"]).str.strip()

# 3) โหลดโมเดล (ถ้ายังไม่มี)
MODEL = "mrp/simcse-model-m-bert-thai-cased"
model = SentenceTransformer(MODEL)

# 4) Encode ใหม่ให้ “สอดคล้องกับ DataFrame นี้เท่านั้น”
emb_trip_name  = model.encode(OTA_df["Trip_Name_n"].tolist(),    convert_to_numpy=True, normalize_embeddings=True, batch_size=128, show_progress_bar=True)
emb_trip_addr  = model.encode(OTA_df["Trip_Address_n"].tolist(),  convert_to_numpy=True, normalize_embeddings=True, batch_size=128, show_progress_bar=True)
emb_trip_key   = model.encode(OTA_df["Trip_Key"].tolist(),        convert_to_numpy=True, normalize_embeddings=True, batch_size=128, show_progress_bar=True)

emb_hotel_name = model.encode(hotel["Hotel_Name_n"].tolist(),     convert_to_numpy=True, normalize_embeddings=True, batch_size=128, show_progress_bar=True)
emb_hotel_addr = model.encode(hotel["Hotel_Address_n"].tolist(),   convert_to_numpy=True, normalize_embeddings=True, batch_size=128, show_progress_bar=True)
emb_hotel_key  = model.encode(hotel["Hotel_Key"].tolist(),         convert_to_numpy=True, normalize_embeddings=True, batch_size=128, show_progress_bar=True)

# 5) ตรวจ alignment
assert emb_trip_key.shape[0]  == len(OTA_df), f"trip mismatch: emb={emb_trip_key.shape[0]} vs df={len(OTA_df)}"
assert emb_hotel_key.shape[0] == len(hotel),  f"hotel mismatch: emb={emb_hotel_key.shape[0]} vs df={len(hotel)}"


Batches:   0%|          | 0/4 [00:00<?, ?it/s]

Batches:   0%|          | 0/4 [00:00<?, ?it/s]

Batches:   0%|          | 0/4 [00:00<?, ?it/s]

Batches:   0%|          | 0/11 [00:00<?, ?it/s]

Batches:   0%|          | 0/11 [00:00<?, ?it/s]

Batches:   0%|          | 0/11 [00:00<?, ?it/s]

In [44]:
#Test
QUERY = "กลับ"
TOPK  = 5
WEIGHTS = dict(key=0.5, name=0.3, addr=0.2)

# 1) เลือกแถว OTA ที่มีคำว่า 'กลับ'
mask = OTA_df["Trip_Name_n"].astype(str).str.contains(QUERY, na=False) | \
       OTA_df["name"].astype(str).str.contains(QUERY, na=False)
if not mask.any():
    raise ValueError(f"ไม่พบที่พักใน OTA_df ที่มีคำว่า '{QUERY}'")

trip_pos   = int(np.flatnonzero(mask)[0])
trip_row   = OTA_df.iloc[trip_pos]
trip_title = str(trip_row.get("name", f"OTA_idx_{trip_pos}"))
trip_addr  = str(trip_row.get("address", ""))

# 2) คำนวณความคล้ายกับฝั่ง hotel
sim_key  = cosine_similarity(emb_trip_key[trip_pos:trip_pos+1],  emb_hotel_key).ravel()
sim_name = cosine_similarity(emb_trip_name[trip_pos:trip_pos+1], emb_hotel_name).ravel()
sim_addr = cosine_similarity(emb_trip_addr[trip_pos:trip_pos+1], emb_hotel_addr).ravel()
score = WEIGHTS['key']*sim_key + WEIGHTS['name']*sim_name + WEIGHTS['addr']*sim_addr

# 3) เลือก Top-5 ตามคะแนนรวม
order = np.argsort(-score)[:TOPK]
result = hotel.iloc[order].copy()
result["sim_key"]  = sim_key[order]
result["sim_name"] = sim_name[order]
result["sim_addr"] = sim_addr[order]
result["score"]    = score[order]

# 4) แสดงผลลัพธ์พร้อมชื่อ OTA_df
cols = ["Hotel_Name", "Hotel_Address","Hotel_Key" ,"sim_key", "sim_name", "sim_addr", "score"]

print(f"📍 OTA ชื่อ: {trip_title}")
print(f"🏠 OTA ที่อยู่: {trip_addr}")
print(f"\nเพื่อนบ้านของ OTA นี้ (Top-{TOPK} โรงแรมที่ใกล้ที่สุด):")
display(result[cols].sort_values("score", ascending=False).reset_index(drop=True))


📍 OTA ชื่อ: โรงแรมกลับ
🏠 OTA ที่อยู่: 22/1 ถ.เพชรบุรี ซ.13, เขตราชเทวี, กรุงเทพฯ, 10400

เพื่อนบ้านของ OTA นี้ (Top-5 โรงแรมที่ใกล้ที่สุด):


,Hotel_Name,Hotel_Address,Hotel_Key,sim_key,sim_name,sim_addr,score
0,กลับโฮเทล,22/1 ซ.สมประสงค์ 2 แขวงถนนพญาไท เขตราชเทวี จ.ก...,กลับ 22/1ซ.สมประสงค์2แขวงถนนพญาไทเขตราชเทวีกรุ...,0.418986,1.000000,0.430295,0.595552
1,โรงแรม เบด บาย ท่าพระ 1,"112,114,116,118,120,122,130 ถ.เพชรเกษม แขวงปาก...",เบดบายท่าพระ1 112114116118120122130เพชรเกษมแขว...,0.537385,0.064248,0.346129,0.357192
2,โรงแรม ลีลา ออร์คิด,833 ถ.เพชรบุรี แขวงถนนพญาไท เขตราชเทวี จ.กรุงเ...,ลีลาออร์คิด 833เพชรบุรีแขวงถนนพญาไทเขตราชเทวีก...,0.398360,0.212709,0.442567,0.351506
3,เวลาดี โกลว์ ประตูน้ำ,919 ถ.เพชรบุรี แขวงถนนพญาไท เขตราชเทวี จ.กรุงเ...,เวลาดีโกลว์ประตูน้ํา 919เพชรบุรีแขวงถนนพญาไทเข...,0.444825,0.131555,0.419549,0.345789
4,คิทสช์ โฮเทล,156/21 ถ.เพชรบุรี แขวงถนนเพชรบุรี เขตราชเทวี จ...,คิทสช์ 156/21เพชรบุรีแขวงถนนเพชรบุรีเขตราชเทวี...,0.518524,0.020127,0.358035,0.336907


In [ ]:
ิิิ

In [49]:
import numpy as np
import pandas as pd
from sklearn.neighbors import NearestNeighbors

# ---------------------------
# พารามิเตอร์
# ---------------------------
K = 30  # จำนวนผู้สมัครต่อแถว OTA เพื่อรี-สกอร์ (ปรับได้ 20–50)
WEIGHTS = dict(key=0.5, name=0.3, addr=0.2)  # น้ำหนักคะแนนรวม
SCORE_THRESH = None  # ใส่ค่าเช่น 0.55 ถ้าต้องการคัดเฉพาะแมตช์ที่มั่นใจ

# ---------------------------
# 0) ตรวจ alignment พื้นฐาน
# ---------------------------
assert emb_trip_key.shape[0]   == len(OTA_df), "emb_trip_key กับ OTA_df ไม่เท่ากัน"
assert emb_hotel_key.shape[0]  == len(hotel),  "emb_hotel_key กับ hotel ไม่เท่ากัน"
assert emb_trip_name.shape == emb_trip_key.shape
assert emb_trip_addr.shape == emb_trip_key.shape
assert emb_hotel_name.shape == emb_hotel_key.shape
assert emb_hotel_addr.shape == emb_hotel_key.shape

n_trip = len(OTA_df)

# ---------------------------
# 1) คัดผู้สมัคร top-K ด้วยคีย์รวม (เร็ว/สเกลดี)
# ---------------------------
nn = NearestNeighbors(n_neighbors=K, metric="cosine")
nn.fit(emb_hotel_key)
dist_k, idx_k = nn.kneighbors(emb_trip_key, return_distance=True)    # dist_k: (n_trip, K)

# cosine similarity ของคีย์รวม
sim_key_k = 1.0 - dist_k                                             # (n_trip, K)

# ---------------------------
# 2) คำนวณ sim_name / sim_addr เฉพาะบนผู้สมัคร แล้วรวมคะแนน
#    ใช้เวคเตอร์ไรซ์ (ไม่วนลูป) เพื่อความเร็ว
# ---------------------------
# รูปทรงที่ต้องการ:
#   emb_trip_name[:, None, :] -> (n_trip, 1, d)
#   emb_hotel_name[idx_k]     -> (n_trip, K, d)
# แล้ว dot product ตามแกนสุดท้าย
sim_name_k = (emb_trip_name[:, None, :] * emb_hotel_name[idx_k]).sum(axis=2)  # (n_trip, K)
sim_addr_k = (emb_trip_addr[:, None, :] * emb_hotel_addr[idx_k]).sum(axis=2)  # (n_trip, K)

score_k = (WEIGHTS['key']  * sim_key_k +
           WEIGHTS['name'] * sim_name_k +
           WEIGHTS['addr'] * sim_addr_k)                                     # (n_trip, K)

# ---------------------------
# 3) เลือกผู้ชนะ (คะแนนรวมสูงสุด) ต่อแถว OTA
# ---------------------------
best_pos        = score_k.argmax(axis=1)                         # (n_trip,)
best_hotel_idx  = idx_k[np.arange(n_trip), best_pos]            # ดัชนี hotel ที่ชนะ
best_score      = score_k[np.arange(n_trip), best_pos]
best_sim_key    = sim_key_k[np.arange(n_trip), best_pos]
best_sim_name   = sim_name_k[np.arange(n_trip), best_pos]
best_sim_addr   = sim_addr_k[np.arange(n_trip), best_pos]

# ---------------------------
# 4) สร้างผลลัพธ์: แสดง "แค่ candidate ที่คะแนนสูงสุด" ต่อแถว OTA
# ---------------------------
best_hotels = hotel.iloc[best_hotel_idx].reset_index(drop=True)

out_cols = [
    "OTA_Index", "OTA_Name", "OTA_Address", "Trip_Key",
    "Match_Hotel_Name", "Match_Hotel_Address", "Match_Hotel_Key",
    "sim_key", "sim_name", "sim_addr", "score"
]

result_all = pd.DataFrame({
    "OTA_Index": np.arange(n_trip),
    "OTA_Name": OTA_df.get("name", pd.Series([""]*n_trip)),
    "OTA_Address": OTA_df.get("address", pd.Series([""]*n_trip)),
    "Trip_Key": OTA_df.get("Trip_Key", pd.Series([""]*n_trip)),
    "Match_Hotel_Name": best_hotels["Hotel_Name"].values,
    "Match_Hotel_Address": best_hotels["Hotel_Address"].values,
    "Match_Hotel_Key": best_hotels.get("Hotel_Key", pd.Series([np.nan]*n_trip)).values,
    "sim_key": best_sim_key,
    "sim_name": best_sim_name,
    "sim_addr": best_sim_addr,
    "score": best_score,
})

# (ถ้ามี Hotel_Ref ใน hotel)
if "Hotel_Ref" in best_hotels.columns:
    result_all["Match_Hotel_Ref"] = best_hotels["Hotel_Ref"].values

# ---------------------------
# 5) (ตัวเลือก) กรองด้วย threshold ถ้าต้องการโชว์เฉพาะแมตช์ที่มั่นใจ
# ---------------------------
if SCORE_THRESH is not None:
    result_view = result_all[result_all["score"] >= SCORE_THRESH].reset_index(drop=True)
else:
    result_view = result_all.copy()

# ---------------------------
# 6) แสดงผล: เฉพาะ "candidate คะแนนรวมสูงสุด" ต่อแถว OTA
# ---------------------------
display(result_view[out_cols].sort_values("score", ascending=False).reset_index(drop=True))


,OTA_Index,OTA_Name,OTA_Address,Trip_Key,Match_Hotel_Name,Match_Hotel_Address,Match_Hotel_Key,sim_key,sim_name,sim_addr,score
0,66,โรงแรมเซินเจิ้น ทาวเวอร์ กรุงเทพฯ,"3390 ถนน เพชรบุรีตัดใหม่ แขวงบางกะปิ, เขตห้วยข...",เซินเจิ้นทาวเวอร์ 3390เพชรบุรีตัดใหม่แขวงบางกะ...,โรงแรม เซินเจิ้น ทาวเวอร์,3390 ถ.เพชรบุรีตัดใหม่ แขวงบางกะปิ เขตห้วยขวาง...,เซินเจิ้นทาวเวอร์ 3390เพชรบุรีตัดใหม่แขวงบางกะ...,1.000000,1.000000,1.000000,1.000000
1,192,โรงแรมซีซั่น เพลส,"61/15 ถ.เพชรเกษม แขวงวัดท่าพระ, เขตบางกอกใหญ่,...",ซีซั่นเพลส 61/15เพชรเกษมแขวงวัดท่าพระเขตบางกอก...,โรงแรมซีซั่น เพลส,61/15 ถ.เพชรเกษม แขวงวัดท่าพระ เขตบางกอกใหญ่ จ...,ซีซั่นเพลส 61/15เพชรเกษมแขวงวัดท่าพระเขตบางกอก...,1.000000,1.000000,1.000000,1.000000
2,307,โคโคเทล กรุงเทพ สุขุมวิท 50,"11 ซ. สุขุมวิท 50 แขวงพระโขนง, เขตคลองเตย, กรุ...",โคโคเทลสุขุมวิท50 11ซ.สุขุมวิท50แขวงพระโขนงเขต...,โคโคเทล กรุงเทพ สุขุมวิท 50,11 ซ.สุขุมวิท50 แขวงพระโขนง เขตคลองเตย จ.กรุงเ...,โคโคเทลสุขุมวิท50 11ซ.สุขุมวิท50แขวงพระโขนงเขต...,1.000000,1.000000,1.000000,1.000000
3,319,ชามา เย็นอากาศ กรุงเทพ,"69 ถ. เย็นอากาศ แขวงช่องนนทรี, เขตยานนาวา, กรุ...",ชามาเย็นอากาศ 69เย็นอากาศแขวงช่องนนทรีเขตยานนา...,ชามา เย็นอากาศ กรุงเทพฯ,69 ถ.เย็นอากาศ แขวงช่องนนทรี เขตยานนาวา จ.กรุง...,ชามาเย็นอากาศ 69เย็นอากาศแขวงช่องนนทรีเขตยานนา...,1.000000,1.000000,1.000000,1.000000
4,83,ชามา เย็นอากาศ กรุงเทพ,"69 ถ. เย็นอากาศ แขวงช่องนนทรี, เขตยานนาวา, กรุ...",ชามาเย็นอากาศ 69เย็นอากาศแขวงช่องนนทรีเขตยานนา...,ชามา เย็นอากาศ กรุงเทพฯ,69 ถ.เย็นอากาศ แขวงช่องนนทรี เขตยานนาวา จ.กรุง...,ชามาเย็นอากาศ 69เย็นอากาศแขวงช่องนนทรีเขตยานนา...,1.000000,1.000000,1.000000,1.000000
...,...,...,...,...,...,...,...,...,...,...,...
495,458,วินด์แฮม แบงค็อก ศูนย์ประชุมแห่งชาติสิริกิติ์,"388 ซ.ริมคลองไผ่สิงห์โต, ถ.พระราม 4, เขตคลองเต...",วินด์แฮมแบงค็อกศูนย์ประชุมแห่งชาติสิริกิติ์ 38...,แกรนด์ เซนเตอร์ พอยต์ เทอร์มินอล 21,2 ซ.สุขุมวิท 19 (วัฒนา) แขวงคลองเตยเหนือ เขตวั...,แกรนด์เซนเตอร์พอยต์เทอร์มินอล21 2ซ.สุขุมวิท19(...,0.355902,0.303594,0.215095,0.312048
496,495,ณ ตะนาว 1969,"323 ถ.ตะนาว เสาชิงช้า, เขตพระนคร, กรุงเทพฯ, 10200",ณตะนาว1969 323ตะนาวเสาชิงช้าเขตพระนครกรุงเทพมห...,สำราญเพลส โฮเทล,302 ถ.เพชรบุรี แขวงถนนเพชรบุรี เขตราชเทวี จ.กร...,สําราญเพลส 302เพชรบุรีแขวงถนนเพชรบุรีเขตราชเทว...,0.442237,0.048815,0.370131,0.309790
497,182,โรงแรมเอเชียกรุงเทพ,"269 ถ.พ, เขตราชเทวี, กรุงเทพฯ, 10400",เอเชีย 269พเขตราชเทวีกรุงเทพมหานคร10400,ชิวแลกซ์ รีสอร์ท โฮเทล,274 ตรอกวัดตรีทศเทพ แขวงบ้านพานถม เขตพระนคร จ....,ชิวแลกซ์รีสอร์ท 274ตรอกวัดตรีทศเทพแขวงบ้านพานถ...,0.377491,0.129902,0.389493,0.305615
498,421,อวาแลน พญาไท airport link/BTS,AVALAN ซอยติดรางรถไฟ ผ่านรางรถไฟแล้วเลี้ยวซ้าย...,อวาแลนพญาไทairportlinkbts avalanซอยติดรางรถไฟผ...,โรงแรม แอทนเรศ,213 ตรอกพุทธโอสถ แขวงสี่พระยา เขตบางรัก จ.กรุง...,แอทนเรศ 213ตรอกพุทธโอสถแขวงสี่พระยาเขตบางรักกร...,0.307355,0.302209,0.293942,0.303129


In [60]:
t_low  = 0.4
t_high = 0.7

def label_score(x):
    if x < t_low:
        return "not_match"
    elif x < t_high:
        return "possible"
    else:
        return "match"

result_view["label"] = result_view["score"].apply(label_score)


In [63]:
result_view[['OTA_Name','Trip_Key','Match_Hotel_Name','Match_Hotel_Key', 'score', 'label']].drop_duplicates().sort_values('score', ascending=False)

,OTA_Name,Trip_Key,Match_Hotel_Name,Match_Hotel_Key,score,label
192,โรงแรมซีซั่น เพลส,ซีซั่นเพลส 61/15เพชรเกษมแขวงวัดท่าพระเขตบางกอก...,โรงแรมซีซั่น เพลส,ซีซั่นเพลส 61/15เพชรเกษมแขวงวัดท่าพระเขตบางกอก...,1.000000,match
66,โรงแรมเซินเจิ้น ทาวเวอร์ กรุงเทพฯ,เซินเจิ้นทาวเวอร์ 3390เพชรบุรีตัดใหม่แขวงบางกะ...,โรงแรม เซินเจิ้น ทาวเวอร์,เซินเจิ้นทาวเวอร์ 3390เพชรบุรีตัดใหม่แขวงบางกะ...,1.000000,match
301,ลิฟ เอเบิล,ลิฟเอเบิล 89/9รางน้ําแขวงถนนพญาไทเขตราชเทวีกรุ...,โรงแรม ลิฟเอเบิล,ลิฟเอเบิล 89/9รางน้ําแขวงถนนพญาไทเขตราชเทวีกรุ...,1.000000,match
349,เดอะ พลิมเพลส โฮเทล,เดอะพลิมเพลส 3338พหลโยธินแขวงจอมพลเขตจตุจักรกร...,โรงแรมเดอะพลิมเพลส,เดอะพลิมเพลส 3338พหลโยธินแขวงจอมพลเขตจตุจักรกร...,1.000000,match
131,ม็อกซี่ แบงคอก ราชประสงค์,ม็อกซี่แบงคอกราชประสงค์ 111ราชดําริห์แขวงลุมพิ...,โรงแรม ม็อกซี่ แบงคอก ราชประสงค์,ม็อกซี่แบงคอกราชประสงค์ 111ราชดําริห์แขวงลุมพิ...,1.000000,match
...,...,...,...,...,...,...
458,วินด์แฮม แบงค็อก ศูนย์ประชุมแห่งชาติสิริกิติ์,วินด์แฮมแบงค็อกศูนย์ประชุมแห่งชาติสิริกิติ์ 38...,แกรนด์ เซนเตอร์ พอยต์ เทอร์มินอล 21,แกรนด์เซนเตอร์พอยต์เทอร์มินอล21 2ซ.สุขุมวิท19(...,0.312048,not_match
495,ณ ตะนาว 1969,ณตะนาว1969 323ตะนาวเสาชิงช้าเขตพระนครกรุงเทพมห...,สำราญเพลส โฮเทล,สําราญเพลส 302เพชรบุรีแขวงถนนเพชรบุรีเขตราชเทว...,0.309790,not_match
182,โรงแรมเอเชียกรุงเทพ,เอเชีย 269พเขตราชเทวีกรุงเทพมหานคร10400,ชิวแลกซ์ รีสอร์ท โฮเทล,ชิวแลกซ์รีสอร์ท 274ตรอกวัดตรีทศเทพแขวงบ้านพานถ...,0.305615,not_match
421,อวาแลน พญาไท airport link/BTS,อวาแลนพญาไทairportlinkbts avalanซอยติดรางรถไฟผ...,โรงแรม แอทนเรศ,แอทนเรศ 213ตรอกพุทธโอสถแขวงสี่พระยาเขตบางรักกร...,0.303129,not_match


In [64]:
result_view.label.value_counts()

,count
label,
possible,275
match,171
not_match,54
